In [1]:
import pandas as pd #імпортуємо бібліотеку pandas
import numpy as np #імпортуємо бібліотеку numpy
from sklearn import preprocessing, datasets, linear_model, model_selection #імпортуємо потрібні модулі з бібліотеки sklearn
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, mean_absolute_error #імпортуємо потрібні функції з бібліотеки sklearn.metrics
from cmath import sqrt #імпортуємо функцію sqrt з бібліотеки cmath

In [2]:
dataframe=pd.read_csv('../Dataset.csv') #зчитуємо в змінну dataframe дані з заданого файлу
dataframe #виводимо дані на екран
print(dataframe.filter('ad_ltv_day3','app_iap_ltv_day3','target_full_ltv_day30',axis=1))

KeyError: ('ad_ltv_day3', 'app_iap_ltv_day3', 'target_full_ltv_day30')

Перевіримо чи є пропущені дані

In [5]:

dataframe.isna().any(axis=1).sum() #порахуємо і виведемо кількість пропущених даних

1374

Бачимо, що користувачів з пропущеними даними всього 1374. Це неймовірно мало порівняно з загальною кількістю користувачів 1467832. Тому ми викинемо ці рядки. Також викинемо останні чотири стовпчики з доходами з користувача і створимо з них ще один датафрейм. 

In [6]:
dataframe.drop(dataframe.index[dataframe.isna().any(axis=1)].tolist(),axis=0,inplace=True)
income_dataframe=dataframe.filter(['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1)
dataframe.drop(columns=['target_sub_ltv_day30','target_iap_ltv_day30','target_ad_ltv_day30','target_full_ltv_day30'],axis=1,inplace=True)
dataframe

,total_sessions_day0,total_sessions_day1,total_sessions_day3,total_sessions_day7,chapters_finished_day0,chapters_finished_day1,chapters_finished_day3,chapters_finished_day7,chapters_opened_day0,chapters_opened_day1,...,app_iap_ltv_day0,app_iap_ltv_day1,app_iap_ltv_day3,media_source,install_date,country_code,ad_ltv_day0,ad_ltv_day1,ad_ltv_day3,platform
0,1.0,1.0,1.0,1.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,SOURCE_23,2021-12-03,COUNTRY_135,0.000000,0.000000,0.000000,ios
1,1.0,1.0,1.0,1.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,SOURCE_2,2021-12-03,COUNTRY_141,0.000000,0.000000,0.000000,android
2,1.0,2.0,2.0,2.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,SOURCE_17,2021-12-03,COUNTRY_141,0.000000,0.000000,0.000000,android
3,1.0,1.0,1.0,1.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,SOURCE_23,2021-12-03,COUNTRY_141,0.000000,0.000000,0.000000,ios
4,1.0,1.0,1.0,1.0,0,0,0,0,0,0,...,0.000000,0.000000,0.000000,SOURCE_1,2021-12-03,COUNTRY_203,0.000000,0.000000,0.000000,android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1467827,11.0,11.0,11.0,11.0,9,9,9,9,10,10,...,4.174198,4.174198,4.174198,SOURCE_17,2021-12-30,COUNTRY_141,0.037158,0.037158,0.037158,ios
1467828,2.0,3.0,6.0,7.0,5,9,11,13,6,10,...,4.899572,7.841280,9.404430,SOURCE_23,2021-12-30,COUNTRY_141,0.039256,0.039256,0.091267,ios
1467829,3.0,5.0,11.0,13.0,5,9,15,18,5,10,...,4.036153,8.875894,13.928170,SOURCE_23,2021-12-30,COUNTRY_141,0.000000,0.000000,0.000000,ios
1467830,5.0,5.0,5.0,5.0,24,24,24,24,24,24,...,7.402224,7.402224,7.402224,SOURCE_4,2021-12-30,COUNTRY_141,0.177550,0.177550,0.177550,ios


In [7]:
income_dataframe #виведемо таблицю з чотирьма фічами які вказують доходи

,target_sub_ltv_day30,target_iap_ltv_day30,target_ad_ltv_day30,target_full_ltv_day30
0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...
1467827,0.000000,4.174198,0.037158,4.211356
1467828,4.705161,10.967580,0.091267,15.764008
1467829,0.000000,14.938221,0.000000,14.938221
1467830,0.000000,7.402224,0.177550,7.579774


In [ ]:
Приберемо з таблиці категоріальні(нечисельні) дані

In [8]:
categorical_dataframe=dataframe.filter(['platform','media_source','install_date', 'country_code'],axis=1)
dataframe.drop(columns=['platform','media_source','install_date', 'country_code'],axis=1,inplace=True) #Приберемо з таблиці категоріальні(нечисельні) дані

In [9]:
categorical_dataframe #Виведемо таблицю з категоріальними даними

,platform,media_source,install_date,country_code
0,ios,SOURCE_23,2021-12-03,COUNTRY_135
1,android,SOURCE_2,2021-12-03,COUNTRY_141
2,android,SOURCE_17,2021-12-03,COUNTRY_141
3,ios,SOURCE_23,2021-12-03,COUNTRY_141
4,android,SOURCE_1,2021-12-03,COUNTRY_203
...,...,...,...,...
1467827,ios,SOURCE_17,2021-12-30,COUNTRY_141
1467828,ios,SOURCE_23,2021-12-30,COUNTRY_141
1467829,ios,SOURCE_23,2021-12-30,COUNTRY_141
1467830,ios,SOURCE_4,2021-12-30,COUNTRY_141


In [ ]:
Розділимо наші вхідні дані на ті, на яких наш алгоритм буде тренуватись і на ті, на яких буде тестуватись(тренувальні дані-80%, тестувальні-20%)

In [10]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)

In [ ]:
Створимо обєкт класу LinearRegression


In [11]:
model=linear_model.LinearRegression(fit_intercept=1)

In [ ]:
Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель

In [12]:
model.fit(X_train, y_train)
print(model)

LinearRegression(fit_intercept=1)


In [ ]:
Отримаємо значення лінійної регресії

In [13]:
predictions=model.predict(X_test)

In [14]:
predictions

array([ 1.14777913e-02,  1.25810325e-03, -1.07548842e-02, ...,
        1.88554626e-02,  6.23437582e-05,  1.25810325e-03])

In [ ]:
Знайдемо середню абсолютну похибку лінійної регресії(MAE)

In [15]:
mean_absolute_error(y_test,predictions)

0.13536579997892909

In [ ]:
Знайдемо середньоквадратичну похибку лінійної регресії(RMSE)

In [16]:
sqrt(mean_squared_error(y_test,predictions))

(1.0979369801852592+0j)

In [ ]:
Знайдемо середньоквадратичну похибку лінійної регресії(MAPE)

In [17]:
mean_absolute_percentage_error(y_test,predictions)

96870813821756.7

In [ ]:
Запишемо масив неважливих на наш погляд метрик

In [18]:
not_needed_features=['ad_ltv_day3', 'chapters_finished_day0', 'chapters_finished_day1', 'chapters_finished_day7', 'chapters_opened_day3', 'chapters_opened_session3', 'chapters_opened_session9',  'diamonds_received_day1', 'diamonds_received_day7', 'tickets_spent_day3']
len(not_needed_features)

10

In [19]:
col=list(dataframe.columns.values)
print(col)#Випишемо в маси назви всіх метрик які ми поки використовуємо(крім доходних та категоріальних метрик)
col=['ad_ltv_day3','app_iap_ltv_day3']
#'app_sub_ltv_day0','chapters_closed_day7','chapters_finished_day3'
#col=['tickets_spent_day0','tickets_spent_session1','diamonds_spent_session1','retained_day1','total_sessions_day0']

test_dataframe=dataframe.filter(col,axis='columns')# Отримаємо таблицю з значеннями двох метрик, які мають найбільші модулі коефіцієнтів лінійної регресії 
""" dummies_platform=pd.get_dummies(categorical_dataframe['platform'],drop_first=True)
test_dataframe=pd.concat([test_dataframe,dummies_platform],axis='columns') """
X_train, X_test, y_train, y_test = model_selection.train_test_split(test_dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)#Розділимо наші вхідні дані на ті, на яких наш алгоритм буде тренуватись і на ті, на яких буде тестуватись(тренувальні дані-80%, тестувальні-20%)
model=linear_model.LinearRegression(fit_intercept=1)#Створимо обєкт класу LinearRegression
model.fit(X_train, y_train)#Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель
print(model.coef_)# Виведемо коефіцієнти лінійної регресії
predictions=model.predict(X_test)# Отримаємо значення лінійної регресії
print(mean_absolute_error(y_test,predictions),sqrt(mean_squared_error(y_test,predictions)))# Знайдемо середню абсолютну похибку лінійної регресії(MAE) та середньоквадратичну похибку лінійної регресії(RMSE)

['total_sessions_day0', 'total_sessions_day1', 'total_sessions_day3', 'total_sessions_day7', 'chapters_finished_day0', 'chapters_finished_day1', 'chapters_finished_day3', 'chapters_finished_day7', 'chapters_opened_day0', 'chapters_opened_day1', 'chapters_opened_day3', 'chapters_opened_day7', 'chapters_closed_day0', 'chapters_closed_day1', 'chapters_closed_day3', 'chapters_closed_day7', 'diamonds_received_day0', 'diamonds_received_day1', 'diamonds_received_day3', 'diamonds_received_day7', 'diamonds_spent_day0', 'diamonds_spent_day1', 'diamonds_spent_day3', 'diamonds_spent_day7', 'tickets_spent_day0', 'tickets_spent_day1', 'tickets_spent_day3', 'tickets_spent_day7', 'retained_day1', 'retained_day3', 'retained_day7', 'chapters_finished_session1', 'chapters_finished_session3', 'chapters_finished_session9', 'chapters_opened_session1', 'chapters_opened_session3', 'chapters_opened_session9', 'chapters_closed_session1', 'chapters_closed_session3', 'chapters_closed_session9', 'diamonds_spent_se

In [ ]:
Перетворимо категоріальну метрику дати встановлення на числову і додамо до головної таблиці даних

In [21]:
dummies_date=pd.get_dummies(categorical_dataframe['install_date'],drop_first=True)
dataframe=pd.concat([dataframe,dummies_date],axis='columns')

In [ ]:
Перетворимо категоріальні метрики коду країн, медіа ресурсів та платформи на якій була встановлена гра на числові і додамо до головної таблиці даних

In [18]:
dummies_countries=pd.get_dummies(categorical_dataframe['country_code'],drop_first=True)
dataframe=pd.concat([dataframe,dummies_countries],axis='columns')
dummies_media=pd.get_dummies(categorical_dataframe['media_source'],drop_first=True)
dataframe=pd.concat([dataframe,dummies_media],axis='columns')
dummies_platform=pd.get_dummies(categorical_dataframe['platform'],drop_first=True)
dataframe=pd.concat([dataframe,dummies_platform],axis='columns')

In [ ]:
Розділимо наші вхідні дані на ті, на яких наш алгоритм буде тренуватись і на ті, на яких буде тестуватись(тренувальні дані-80%, тестувальні-20%)

In [ ]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(dataframe,income_dataframe['target_full_ltv_day30'], test_size=0.2)


In [ ]:
Створимо обєкт класу LinearRegression
Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель
Отримаємо значення лінійної регресії


In [ ]:
model2=linear_model.LinearRegression(fit_intercept=1)#Створимо обєкт класу LinearRegression
model2.fit(X_train, y_train)#Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель
predictions2=model2.predict(X_test)#Отримаємо значення лінійної регресії

In [ ]:
Знайдемо середню абсолютну похибку лінійної регресії(MAE)

In [ ]:
mean_absolute_error(y_test,predictions)

0.5150409548871169

In [ ]:
Знайдемо середньоквадратичну похибку лінійної регресії(RMSE)

In [ ]:
sqrt(mean_squared_error(y_test,predictions))

(2.993490411204281+0j)

In [ ]:
Сам не поняв шо це

In [ ]:
for i in range(len(model2.coef_)):
    if abs(model2.coef_[i])<0.1: model2.coef_[i]=0
print(model2.coef_)

[ 0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.15681514 -0.16998328  0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.         -0.10298484 -0.22449847
 -0.35701292  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          1.44043495 -0.20379821
  1.29406347 -0.12771713 -0.44117752  1.67804624 -0.41186012 -2.92851333
  4.85012765  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
 

In [ ]:
Створимо обєкт класу LinearRegression
Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель
Отримаємо значення лінійної регресії


In [ ]:
predictions2=model2.predict(X_test)#Створимо обєкт класу LinearRegression
print(mean_absolute_error(y_test,predictions))#Передамо у обєкт класу лінійної регресії дані, на яких тренуватимемо модель
print(sqrt(mean_squared_error(y_test,predictions)))#Отримаємо значення лінійної регресії

0.5362737257560155
(2.993581283512038+0j)
